In [1]:
from __future__ import print_function
import argparse
import sys
import time 
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from datasets import *
from net.model import *

import matplotlib.pyplot as plt
import cv2
import os

In [2]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ['CUDA_VISIBLE_DEVICES'] = '3,4'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## PKU-sketech dataset

In [3]:
from config import Config_pku
from engine.pku_market_trainer import *
pku_args = Config_pku()

In [4]:
train_set, test_color_set, test_sketch_set = make_data_loader_pku(pku_args)

n_pku_clc = len(np.unique(train_set.train_data['sketch_labels'])) # number of person in training set
n_test_color = len(test_color_set.color_labels) #number of instance in test color set
n_test_sketch = len(test_sketch_set.sketch_labels) #number of instance in test sketch set

pku_train_loader = data.DataLoader(train_set, batch_size=pku_args.batch_size, shuffle=True, num_workers=pku_args.num_workers, drop_last=True)
pku_test_color_loader = data.DataLoader(test_color_set, batch_size=pku_args.test_batch, shuffle=False, num_workers=pku_args.num_workers)
pku_test_sketch_loader = data.DataLoader(test_sketch_set, batch_size=pku_args.test_batch, shuffle=False, num_workers=pku_args.num_workers)

test_color_labels = test_color_set.color_labels
test_sketch_labels = test_sketch_set.sketch_labels

nquery = n_test_sketch
ngall = n_test_color

query_loader = pku_test_sketch_loader
gall_loader = pku_test_color_loader 
query_label = test_sketch_labels
gall_label = test_color_labels

==> Loading data..


In [5]:
Embed_net_1 = Baseline(pretrain_choice=None)
Embed_net_2 = Baseline(pretrain_choice=None)
Classify_net = C_net(2048,n_pku_clc,0.7)
A_net = Attribute_net(dim=2048, n_att=8)

Embed_net_1.to(device)
Embed_net_2.to(device)
Classify_net.to(device)
A_net.to(device)

checkpoint = torch.load(pku_args.model_path+'pku_best_96.t')
Embed_net_1.load_state_dict(checkpoint['Embed_net_1'])
Embed_net_2.load_state_dict(checkpoint['Embed_net_2']) 
Classify_net.load_state_dict(checkpoint['Classify_net']) 

Embed_net_1.eval()
Embed_net_2.eval()
Classify_net.eval()

C_net(
  (bottleneck): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (classifier): Sequential(
    (0): Dropout(p=0.7)
    (1): Linear(in_features=2048, out_features=150, bias=True)
  )
)

In [6]:
print ('Extracting Query Feature...')
ptr = 0
query_feat = np.zeros((nquery, 2048))
with torch.no_grad():
    for batch_idx, (img, label ) in enumerate(query_loader):
        batch_num = img.size(0)
        img = img.cuda()
        _,feat = Embed_net_2(img)
        feat = Classify_net(feat)
        feat = F.normalize(feat, p=2, dim=1) # normorlize features
        query_feat[ptr:ptr+batch_num,: ] = feat.detach().cpu().numpy()
        ptr = ptr + batch_num

        
print ('Extracting Gallery Feature...')
ptr = 0
gall_feat = np.zeros((ngall, 2048))
with torch.no_grad():
    for batch_idx, (img, label ) in enumerate(gall_loader):
        batch_num = img.size(0)
        img = img.cuda()
        _,feat = Embed_net_1(img)
        feat = Classify_net(feat)
        feat = F.normalize(feat, p=2, dim=1) # normorlize features
        gall_feat[ptr:ptr+batch_num,: ] = feat.detach().cpu().numpy()
        ptr = ptr + batch_num 

Extracting Query Feature...
Extracting Gallery Feature...


In [7]:
# compute the similarity
distmat  = np.matmul(query_feat, np.transpose(gall_feat))
cmc, mAP = eval_pku(-distmat, query_label, gall_label)
print(f'CMC from 1 to 20:{cmc}')
print(f'mAP:{mAP}')

CMC from 1 to 20:[0.96 0.98 0.98 0.98 1.   1.   1.   1.   1.   1.   1.   1.   1.   1.
 1.   1.   1.   1.   1.   1.  ]
mAP:0.9430336134453782


# Shoe

In [8]:
from config import Config_sbir
from engine.zap_sbir_trainer import *

In [9]:
sbir_args = Config_sbir()

In [10]:
train_set, test_color_set, test_sketch_set = make_data_loader_sbir(sbir_args)

n_sbir_clc = len(np.unique(train_set.train_data['labels'])) 
n_test_color = len(test_color_set.color_labels) 
n_test_sketch = len(test_sketch_set.sketch_labels) 


sbir_train_loader = data.DataLoader(train_set, batch_size=sbir_args.batch_size, shuffle=True, num_workers=sbir_args.num_workers, drop_last=False)
sbir_test_color_loader = data.DataLoader(test_color_set, batch_size=sbir_args.test_batch, shuffle=False, num_workers=sbir_args.num_workers)
sbir_test_sketch_loader = data.DataLoader(test_sketch_set, batch_size=sbir_args.test_batch, shuffle=False, num_workers=sbir_args.num_workers)

test_color_labels = np.array(test_color_set.color_labels)
test_sketch_labels = np.array(test_sketch_set.sketch_labels)

nquery = n_test_sketch
ngall = n_test_color
query_loader = sbir_test_sketch_loader
gall_loader = sbir_test_color_loader 
query_label = test_sketch_labels
gall_label = test_color_labels

==> Loading data..


In [11]:
Embed_net_1 = Baseline(pretrain_choice=None)
Embed_net_2 = Baseline(pretrain_choice=None)
Classify_net = C_net(sbir_args.low_dim,n_sbir_clc, 0.7)
A_net = Attribute_net(dim=sbir_args.low_dim, n_att=sbir_args.num_att)
D_net = Domain_net(dim=sbir_args.low_dim)

Embed_net_1.to(device)
Embed_net_2.to(device)
Classify_net.to(device)
A_net.to(device)

Embed_net_1 = nn.DataParallel(Embed_net_1)
Embed_net_2 = nn.DataParallel(Embed_net_2)
Classify_net = nn.DataParallel(Classify_net)
A_net = nn.DataParallel(A_net)

checkpoint = torch.load(sbir_args.model_path+'sbir_best_69.t')
Embed_net_1.load_state_dict(checkpoint['Embed_net_1'])
Embed_net_2.load_state_dict(checkpoint['Embed_net_2'])   

Embed_net_1.eval()
Embed_net_2.eval()

DataParallel(
  (module): Baseline(
    (base): SENet(
      (layer0): Sequential(
        (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
      )
      (layer1): Sequential(
        (0): SEResNeXtBottleneck(
          (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=

In [12]:
print ('Extracting Query Feature...')
ptr = 0
query_feat = np.zeros((nquery, 2048))
with torch.no_grad():
    for batch_idx, (img, label ) in enumerate(query_loader):
        batch_num = img.size(0)
        img = img.cuda()
        _,feat = Embed_net_2(img)
        feat = F.normalize(feat, p=2, dim=1) # normorlize features
        query_feat[ptr:ptr+batch_num,: ] = feat.detach().cpu().numpy()
        attributes = A_net(feat)
        ptr = ptr + batch_num         
print ('Extracting Gallery Feature...')
ptr = 0
gall_feat = np.zeros((ngall, 2048))

with torch.no_grad():
    for batch_idx, (img, label ) in enumerate(gall_loader):
        batch_num = img.size(0)
        img = img.cuda()
        _,feat = Embed_net_1(img)
        feat = F.normalize(feat, p=2, dim=1) # normorlize features
        gall_feat[ptr:ptr+batch_num,: ] = feat.detach().cpu().numpy()
        ptr = ptr + batch_num 

# compute the similarity
distmat  = np.matmul(query_feat, np.transpose(gall_feat))


Extracting Query Feature...
Extracting Gallery Feature...


In [13]:
from eval_metrics import eval_sbir
cmc, mAP = eval_sbir(-distmat, query_label, gall_label)
print(f'CMC from 1 to 20:{cmc}')
print(f'mAP:{mAP}')

CMC from 1 to 20:[0.6869565  0.79130435 0.8521739  0.87826085 0.90434784 0.93043476
 0.9478261  0.9478261  0.9478261  0.9478261  0.95652175 0.95652175
 0.95652175 0.9652174  0.9652174  0.9652174  0.9652174  0.9652174
 0.9652174  0.9652174 ]
mAP:0.7802725383916311


# transfer weigh from parrall to single gpu

In [26]:
Embed_net_1 = Baseline(pretrain_choice=None)
Embed_net_2 = Baseline(pretrain_choice=None)
Classify_net = C_net(sbir_args.low_dim,n_sbir_clc, 0.7)
A_net = Attribute_net(dim=sbir_args.low_dim, n_att=sbir_args.num_att)
D_net = Domain_net(dim=sbir_args.low_dim)

Embed_net_1 = nn.DataParallel(Embed_net_1)
Embed_net_2 = nn.DataParallel(Embed_net_2)
Classify_net = nn.DataParallel(Classify_net)
A_net = nn.DataParallel(A_net)
D_net = nn.DataParallel(D_net) 

checkpoint = torch.load(sbir_args.model_path+'zap_p.t')
Embed_net_1.load_state_dict(checkpoint['Embed_net_1'])
Embed_net_2.load_state_dict(checkpoint['Embed_net_2'])   
A_net.load_state_dict(checkpoint['A_net'])
      
state = {
    'Embed_net_1': Embed_net_1.module.state_dict(),
    'Embed_net_2': Embed_net_2.module.state_dict(),
    'Classify_net': Classify_net.module.state_dict(),
    'A_net': A_net.module.state_dict(),
}
torch.save(state, 'zap.t')